In [9]:
import os
import cv2

### Params

In [10]:
video_path = "assets/correct/20250119_145800.mp4"
video_extraction_path = "video_extraction/correct/20250111_092305_frames"
data_output_folder = "data/bench_press"
label_list = ["correct", "error1", "error2"]
primary_label = "correct"
skip_frame_by_s = 0.25 # skip 0.25 second

### Kiểm tra thông số của video

In [11]:
def check_frames_per_second(video_path: str):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Không thể mở video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")

    video_length_in_seconds = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

    print(f"Video length in seconds: {video_length_in_seconds}")

In [12]:
check_frames_per_second(video_path)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.00058466425655
Video length in seconds: 27


### Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder

In [13]:
def extract_images_from_video(video_path: str, output_folder: str, time_step: float = 0.25):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lấy tên file gốc không bao gồm phần mở rộng
    base_name = os.path.splitext(os.path.basename(video_path))[0]

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS): {fps}")

    frame_interval = int(fps * time_step)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            saved_count += 1
            # Định dạng tên file: <tên file gốc>_<số thứ tự dạng 3 chữ số>.jpg
            output_file = os.path.join(output_folder, f"{base_name}_{saved_count:03d}.jpg")
            cv2.imwrite(output_file, frame)

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()


In [14]:
extract_images_from_video(video_path, video_extraction_path, skip_frame_by_s)

Frames per second using video.get(cv2.CAP_PROP_FPS): 30.00058466425655


### Tool

In [15]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QPushButton, QWidget, QVBoxLayout, QHBoxLayout, QCheckBox
from PyQt5.QtGui import QPixmap, QTransform
from PyQt5.QtCore import Qt
import os
import shutil

def copy_images_to_folder(input_file: str, output_folder: str, angle: int):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Đọc ảnh gốc vào QPixmap
    pixmap = QPixmap(input_file)

    # Xoay ảnh theo góc
    transform = QTransform().rotate(angle)
    rotated_pixmap = pixmap.transformed(transform)

    # Tạo đường dẫn đầy đủ cho ảnh đích
    output_file = os.path.join(output_folder, os.path.basename(input_file))

    # Lưu ảnh đã xoay vào thư mục đích
    rotated_pixmap.save(output_file)

def window(input_folder: str, output_folder: str, label_list: list, primary_label: str):
    current_angle = 0

    app = QApplication(sys.argv)
    main_window = QMainWindow()
    main_window.setGeometry(200, 200, 1000, 800)
    main_window.setWindowTitle("Labeling Tool")
    main_window.setStyleSheet("background-color: black;")

    images = os.listdir(input_folder)
    image_count = len(images)
    current_image_index = 0

    central_widget = QWidget()
    main_window.setCentralWidget(central_widget)

    layout = QVBoxLayout()
    central_widget.setLayout(layout)

    # Tạo label hiển thị tên ảnh
    name_pic = QLabel()
    name_pic.setStyleSheet("color: white;")
    layout.addWidget(name_pic)

    # Vẽ ảnh lên màn hình cửa sổ với ảnh từ file
    pixmap = QPixmap(f"{input_folder}/{images[current_image_index]}")
    label = QLabel()
    label.setPixmap(pixmap)
    label.setScaledContents(True)
    label.setFixedSize(500, 600)
    layout.addWidget(label)

    # Tạo các checkbox từ label_list
    checkboxes = []
    for label_name in label_list:
        checkbox = QCheckBox(label_name)
        checkbox.setStyleSheet("color: white;")
        layout.addWidget(checkbox)
        checkboxes.append(checkbox)

        # Thêm sự kiện khi thay đổi trạng thái checkbox
        def on_checkbox_state_changed(state, current_checkbox=checkbox):
            if current_checkbox.text() == primary_label and state == Qt.Checked:
                # Bỏ chọn tất cả các checkbox khác
                for other_checkbox in checkboxes:
                    if other_checkbox != current_checkbox:
                        other_checkbox.setChecked(False)
            
            if current_checkbox.text() != primary_label and state == Qt.Checked:
                # Bỏ chọn primary_label
                for other_checkbox in checkboxes:
                    if other_checkbox.text() == primary_label:
                        other_checkbox.setChecked(False)

        # Kết nối sự kiện stateChanged với hàm xử lý
        checkbox.stateChanged.connect(on_checkbox_state_changed)

    # Tạo label hiển thị trạng thái
    label_status = QLabel()
    label_status.setStyleSheet("color: white;")
    layout.addWidget(label_status)

    def rotate_image(angle):
        nonlocal pixmap
        transform = QTransform().rotate(angle)
        rotated_pixmap = pixmap.transformed(transform)
        label.setPixmap(rotated_pixmap)

    def update_ui():
        nonlocal current_image_index, current_angle
        pixmap.load(f"{input_folder}/{images[current_image_index]}")
        label.setPixmap(pixmap)
        name_pic.setText(f"{current_image_index + 1}/{image_count} - {images[current_image_index]}")
        # Xoay ảnh về góc hiên tại
        rotate_image(current_angle)

    update_ui()

    # Tạo layout ngang cho nút Back và Continue
    button_layout = QHBoxLayout()
    layout.addLayout(button_layout)

    # Tạo button để quay lại ảnh trước
    btn_back = QPushButton("Back")
    btn_back.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_back)

    def btn_back_clicked():
        nonlocal current_image_index
        current_image_index -= 1
        if current_image_index < 0:
            current_image_index = len(images) - 1

        # Set lại trạng thái của checkbox
        for checkbox in checkboxes:
            checkbox.setChecked(False)

        update_ui()

    btn_back.clicked.connect(btn_back_clicked)

    # Tạo button để show ảnh tiếp theo
    btn_continue = QPushButton("Continue")
    btn_continue.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_continue)

    def btn_continue_clicked():
        nonlocal current_image_index
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0

        # Set lại trạng thái của checkbox
        for checkbox in checkboxes:
            checkbox.setChecked(False)

        update_ui()

    btn_continue.clicked.connect(btn_continue_clicked)

    # Tạo button để lưu ảnh
    btn_save = QPushButton("Save and continue (click S or s)")
    btn_save.setStyleSheet("background-color: white;")
    layout.addWidget(btn_save)

    def btn_save_clicked():
        nonlocal current_image_index
        selected_labels = [checkbox.text() for checkbox in checkboxes if checkbox.isChecked()]

        if not selected_labels:
            label_status.setText("No label selected")
            return

        print(f"Copying {images[current_image_index]} to folders: {', '.join(selected_labels)}")

        for label_name in selected_labels:
            copy_images_to_folder(
                f"{input_folder}/{images[current_image_index]}",
                f"{output_folder}/{label_name}",
                current_angle
            )

        label_status.setText(f"Copied {images[current_image_index]} to folders: {', '.join(selected_labels)}")

        # Bỏ chọn tất cả checkbox
        for checkbox in checkboxes:
            checkbox.setChecked(False)

        # Tự động chuyển sang ảnh tiếp theo
        btn_continue_clicked()

    btn_save.clicked.connect(btn_save_clicked)

    # Thêm sự kiện bàn phím
    def key_press_event(event):
        nonlocal current_image_index
        if event.key() in [Qt.Key_S, Qt.Key_S]:
            btn_save_clicked()

    main_window.keyPressEvent = key_press_event

    # Tạo các nút xoay ảnh
    # Tạo button xoay ảnh sang trái
    btn_rotate_left = QPushButton("Rotate Left")
    btn_rotate_left.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_left)

    def btn_rotate_left_clicked():
        nonlocal current_angle
        current_angle -= 90
        if current_angle < 0:
            current_angle = 270

        rotate_image(current_angle)

    btn_rotate_left.clicked.connect(btn_rotate_left_clicked)

    # Tạo button xoay ảnh sang phải
    btn_rotate_right = QPushButton("Rotate Right")
    btn_rotate_right.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_right)

    def btn_rotate_right_clicked():
        nonlocal current_angle
        current_angle += 90
        if current_angle >= 360:
            current_angle = 0

        rotate_image(current_angle)

    btn_rotate_right.clicked.connect(btn_rotate_right_clicked)

    main_window.show()
    try:
        sys.exit(app.exec_())
    except:
        print("Exiting")


: 

In [ ]:
window(video_extraction_path, data_output_folder, label_list, primary_label)

Copying 20250119_145800_001.jpg to folders: error1, error2
Copying 20250119_145800_002.jpg to folders: correct
Copying 20250119_145800_003.jpg to folders: error1, error2
